In [1]:
import os, copy
import numpy as np
import pandas as pd
import seaborn as sns
import pingouin as pg
from scipy import stats

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def rename_subj(sidx):
    if sidx < 100:
        return "sub-{:02d}".format(sidx)
    else:
        return "sub-{:03d}".format(sidx)

In [4]:
def format_p(p):
    if p < .001:
        return "< .001***"
    elif p < .01:
        return f"{p:.3f}**".lstrip('0')
    elif p < .05:
        return f"{p:.3f}*".lstrip('0')
    elif p == 1:
        return "1"
    else:
        return f"{p:.3f}".lstrip('0')

In [5]:
## TYPE-1:
def format_correlation(r, p):
    if p < .001:
        return f"[ {r:.2f}*** ]"
    elif p < .01:
        return f"[ {r:.2f}** ]"
    elif p < .05:
        return f"[ {r:.2f}* ]"
    else:
        return f"{r:.2f}"

In [6]:
# ## TYPE-2:
# def format_correlation(r, p):
#     if p < .001:
#         return f"{r:.2f}***"
#     elif p < .01:
#         return f"{r:.2f}**"
#     elif p < .05:
#         return f"{r:.2f}*"
#     else:
#         return f"{r:.2f}"

In [7]:
def remove_outlier_indices(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    not_outliers = ~ ((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR)))
    
    return not_outliers

In [8]:
change_R_names = {
    'run-02': "1",
    'run-03': "2",
    'run-04': "3",
    'run-05': "4",
    'run-06': "5",
    'run-07': "6",
    'run-08': "7",
    'run-09': "8"    
}

# Load data:

In [9]:
top_dir = "." 

ap_uni = "runs_MNI_3ord_v6_REML"
ap_ppi = "runs_MNI_3ord_v6.PPI_REML"

gfunc = ["", "_gfunc"][1]

roi_list = [
    ["V1+2", "LOC-V12", "Fusiform", "Inf_Temp", "PHC", "PRC", "ERC", "Hippocampus"], 
    ["V1+2", "hLOC+Fus+IT", "MTL", "Hippocampus"], 
    ["V1+2", "LOC-V12", "Fusiform", "Inf_Temp", 
     "PHC", "PRC", "ERC", "Hippocampus", 
     "Hipp-DG+CA4", "Hipp-CA2+3", "Hipp-CA1", "Hipp-Sub"], 
][2]
rois_name = ["Vis_MTL_Hipp", "2.Vis_1.MTL_Hipp", "Vis_MTL_Hipp.Sub"][0]

ppi_seed = "Hippocampus"
ppi_rois = [roi for roi in roi_list if roi != "Hippocampus"]
ppi_rois_name = rois_name.replace("_Hipp", "")

cond_list =  ["Fst", "Snd", "Trd"]

threshold = [27, 21.33][0]
learner_only = 0
remove_outlier = 0

In [10]:
subj_group = ["", "_rapid", "_slow"][0] # for similarity change

In [11]:
## Univariate activity: 

uni_dir = os.path.join(top_dir, f"ROIstats{gfunc}", ap_uni)
uni_data = "maskave.all_ROI_Run_Cond.csv" 
uni_table = pd.read_csv(os.path.join(uni_dir, uni_data), sep=",", header=0)
# uni_table["Y_type"] = "Uni"

uni_conds = set(uni_table.Cond)
uni_table_2 = copy.deepcopy(uni_table)
uni_table = uni_table.query("Cond in @cond_list")

In [12]:
## Functional connectivity:

ppi_dir = os.path.join(top_dir, f"ROIstats{gfunc}", ap_ppi, ppi_seed)
ppi_data = "maskave.all_ROI_Run_Cond.csv" 
ppi_table = pd.read_csv(os.path.join(ppi_dir, ppi_data), sep=",", header=0)
ppi_table = ppi_table.rename(columns={"Y": "PPI"})

# ppi_conds = set(ppi_table.Cond)
ppi_table = ppi_table.query("Cond in @cond_list")

In [13]:
## Combine data of univariate activity & functional connectivity:

data_table = uni_table.merge(ppi_table, how="left", 
                             on=["Subj", "ROI", "Run", "Cond"])

data_table["Run"] = data_table["Run"].map(lambda x: "run-{:02}".format(x))
data_table["Run"] = data_table["Run"].map(change_R_names)
run_list = sorted(list(set(data_table["Run"])))

data_table = data_table.query("ROI in @roi_list")

# data_table["Y"] = data_table["Y"] *100
# data_table["PPI"] = data_table["PPI"] *100

In [14]:
## Behavior scores:

behav_dir = os.path.join(top_dir, "Familiarity_test_performance")
behav_fn = "subj_score_LN.tsv" 
behav_table = pd.read_csv(os.path.join(behav_dir, behav_fn), 
                          sep="\t", header=0)

behav_table["Score"] = behav_table["Score"] * 60

In [15]:
## Combine with Behavior scores:

data_table = data_table.merge(behav_table, how="left", on="Subj")
subj_list = behav_table["Subj"]

In [16]:
# ## Slope (fit univ. activity across time)

# folder = f"[gfunc] {rois_name} (Fst & Snd & Trd)" if gfunc == "_gfunc" else f"{rois_name} (Fst & Snd & Trd)"
# roistat_dir = os.path.join(top_dir, "ROI_analysis", ap_uni, "ANOVA", folder)

# slope_fn = "Slope_per_Subj.csv" 
# slope_table = pd.read_csv(os.path.join(roistat_dir, slope_fn), header=0, index_col=0)

# slope_table["Subj"] = slope_table["Subj"].map(rename_subj)
# slope_table = slope_table.rename(columns={"estimate": "Slope"})

# data_table = data_table.merge(slope_table.loc[:, ["ROI", "Cond", "Subj", "Slope"]], 
#                               how="left", on=["Subj", "ROI", "Cond"])

# # slope_table = slope_table.drop("term", axis=1)
# slope_table = (slope_table
#                .loc[:, ["ROI", "Cond", "Subj", "Slope"]]
#                .merge(behav_table, how="left", on="Subj"))

In [17]:
# ## Try: Compute differences in intensity 

# if 'diff_table' in locals():
#     del diff_table

# for run_1, run_2 in zip(run_list[:-1], run_list[1:]):
#     r_diff = f"{run_2}-{run_1}"
    
#     r1_df = (data_table
#              .query("Run == @run_1")
#              .loc[:, ["Subj", "ROI", "Cond", "Y"]])
#     r1_df["R_diff"] = r_diff
    
#     r2_df = (data_table
#              .query("Run == @run_2")
#              .loc[:, ["Subj", "ROI", "Cond", "Y"]])
#     r2_df["R_diff"] = r_diff
    
#     temp_table = (r2_df
#                   .set_index(["Subj", "ROI", "Cond", "R_diff"])
#                   .subtract(r1_df
#                             .set_index(["Subj", "ROI", "Cond", "R_diff"])))
        
#     if 'diff_table' not in locals():
#         diff_table = temp_table
#     else:
#         diff_table = pd.concat([diff_table, temp_table])
        
# diff_table = diff_table.reset_index()

# r_diff_list = [
#     f"{run_2}-{run_1}" for run_1, run_2 in zip(run_list[:-1], run_list[1:])
# ]

In [18]:
# diff_table = diff_table.merge(behav_table, how="left", on="Subj")

In [19]:
## Target types:

targ_namelist = [
    "fst.pairs", "snd.pairs", "tpl.nonadj", 
    "within.tpl", "rare.connect", "irrelvant", "diff.tpl", 
    "tpl.A", "tpl.B", "tpl.C", "tpl.D", "Fst", "Snd", "Trd"
]

In [20]:
## Similarity changes: 

mvpa_top = os.path.join(top_dir, "PyMVPA")
rdm_meas = f"PDist_{['correlation', 'sqeuclidean'][0]}"

long_DF_file = os.path.join(mvpa_top, f"long_DF_{rdm_meas}.csv")

if not os.path.exists(long_DF_file): 
    smc_DF_file = os.path.join(mvpa_top, 
                               "similarity_matrices", 
                               "tpl_sorted", 
                               "ap-runs_v7.5", 
                               "LSS.Betas", 
                               rdm_meas, 
                               "r01_x_r10", 
                               "all_similarity_changes.csv")
    smc_DF = pd.read_csv(smc_DF_file, header=[0, 1], index_col=0)
    
    targ_list = targ_namelist ###
    select_targ = smc_DF.columns.get_level_values("Targ").isin(targ_list)
    used_smc_DF = smc_DF.loc[:, select_targ]

    used_smc_DF = smc_DF
    long_DF = pd.melt(used_smc_DF.reset_index(), 
                      id_vars="index", 
                      var_name=["ROI", "Targ"], 
                      value_name="Avg_SMC")   
    long_DF["Group"] = [ "rapid" if int(sid.replace("sub-", "")) < 100 else "slow" for sid in long_DF["index"] ]
    long_DF = long_DF.rename(columns={"index": "Subj"})
    long_DF.to_csv(long_DF_file, index=False) 

smc_table = pd.read_csv(long_DF_file, header=0)

In [21]:
if subj_group == "_rapid":
    smc_table = smc_table.query("Group == 'rapid'")
elif subj_group == "_slow":
    smc_table = smc_table.query("Group == 'slow'")
else:
    smc_table = smc_table

In [22]:
## Combine with Behavior scores:

smc_table = smc_table.merge(behav_table, how="left", on="Subj")
subj_list = behav_table["Subj"]

## Correlate all, save (.csv), and plot Heatmaps (.png):

In [23]:
## Mean intensities between hipp & other ROIs:

re_do = 0

if re_do:
    all_cormat, all_annot_mat = {}, {}
    fw, fh = 12, 6
    dv = "Y"
    cond_pairs = [
        ["Fst", "Fst"], 
        ["Snd", "Snd"], 
        ["Trd", "Trd"], 
        ["Fst", "Snd"], 
        ["Fst", "Trd"], 
        ["Snd", "Trd"]
    ]
    
    folder = f"[gfunc] Hipp_x_{ppi_rois_name}" if gfunc == "_gfunc" else f"Hipp_x_{ppi_rois_name}"
    out_dir = os.path.join(top_dir, "ROI_analysis", 
                           ap_uni, "InterSubj_Corr", folder)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)  

    for cond_1, cond_2 in cond_pairs: 
        cond_pair = cond_1+"_x_"+cond_2
        out_csv_fn = f"hipp.{cond_pair}.csv"
        out_fig_fn = f"hipp.{cond_pair}.png"
        
        all_cormat[cond_pair], all_annot_mat[cond_pair] = {}, {}

        for roi in ppi_rois:
            all_cormat[cond_pair][roi], all_annot_mat[cond_pair][roi] = {}, {}

            for run in run_list:
                hipp_Y = data_table.query("ROI == 'Hippocampus' & "+
                                          "Cond == @cond_1 & "+
                                          "Run == @run")[dv]

                roi_Y = data_table.query("ROI == @roi & "+
                                         "Cond == @cond_2 & "+
                                         "Run == @run")[dv]

                r, p = stats.pearsonr(hipp_Y, roi_Y, alternative="two-sided")
                all_cormat[cond_pair][roi][run] = r
                all_annot_mat[cond_pair][roi][run] = format_correlation(r, p)

        cormat = pd.DataFrame(all_cormat[cond_pair]).T
        cormat.to_csv(os.path.join(out_dir, out_csv_fn))

        # -------------------------

        fig, ax = plt.subplots(figsize=(fw, fh), dpi=500) 
        sns.set(style='whitegrid', font_scale=2)
        heatmap = sns.heatmap(
            cormat, vmin=-1, vmax=1, cmap="RdBu_r", cbar=False, 
            annot=pd.DataFrame(all_annot_mat[cond_pair]).T, fmt = "", 
            linewidth=.5 
        )
        ax.set_xticklabels(run_list, rotation=0, fontsize=24)
        ax.set_yticklabels(ppi_rois, rotation=0, fontsize=24)
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, out_fig_fn), 
                    format='png', dpi=200)
        plt.close()

In [24]:
## Cross-run intensity diffs between hipp & other ROIs:

re_do = 0

if re_do:
    all_cormat, all_annot_mat = {}, {}
    fw, fh = 12, 6
    dv = "Y"
#     r_diff_list = [
#         f"{run_2}-{run_1}" for run_1, run_2 in zip(run_list[:-1], run_list[1:])
#     ]
    cond_pairs = [
        ["Fst", "Fst"], 
        ["Snd", "Snd"], 
        ["Trd", "Trd"], 
        ["Fst", "Snd"], 
        ["Fst", "Trd"], 
        ["Snd", "Trd"]
    ]
    
    folder = f"[gfunc] Hipp_x_{ppi_rois_name}" if gfunc == "_gfunc" else f"Hipp_x_{ppi_rois_name}"
    out_dir = os.path.join(top_dir, "ROI_analysis", 
                           ap_uni, "InterSubj_Corr", folder)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)  

    for cond_1, cond_2 in cond_pairs: 
        cond_pair = cond_1+"_x_"+cond_2
        out_csv_fn = f"[Run diff] hipp.{cond_pair}.csv"
        out_fig_fn = f"[Run diff] hipp.{cond_pair}.png"
        
        all_cormat[cond_pair], all_annot_mat[cond_pair] = {}, {}

        for roi in ppi_rois:
            all_cormat[cond_pair][roi], all_annot_mat[cond_pair][roi] = {}, {}

            for r_diff in r_diff_list:
                hipp_Y = diff_table.query("ROI == 'Hippocampus' & "+
                                          "Cond == @cond_1 & "+
                                          "R_diff == @r_diff")[dv]

                roi_Y = diff_table.query("ROI == @roi & "+
                                         "Cond == @cond_2 & "+
                                         "R_diff == @r_diff")[dv]

                r, p = stats.pearsonr(hipp_Y, roi_Y, alternative="two-sided")
                all_cormat[cond_pair][roi][r_diff] = r
                all_annot_mat[cond_pair][roi][r_diff] = format_correlation(r, p)

        cormat = pd.DataFrame(all_cormat[cond_pair]).T
        cormat.to_csv(os.path.join(out_dir, out_csv_fn))

        # -------------------------

        fig, ax = plt.subplots(figsize=(fw, fh)) 
        sns.set(style='whitegrid', font_scale=1.8)
        heatmap = sns.heatmap(
            cormat, vmin=-1, vmax=1, cmap="RdBu_r", cbar=False, 
            annot=pd.DataFrame(all_annot_mat[cond_pair]).T, fmt = "", 
            linewidth=.5 
        )
        ax.set_xticklabels(r_diff_list, rotation=0, fontsize=20)
        ax.set_yticklabels(ppi_rois, rotation=0, fontsize=20)
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, out_fig_fn), 
                    format='png', dpi=200)
        plt.close()

In [25]:
## Behavior scores x Slopes:

re_do = 0

if re_do:
    all_cormat, all_annot_mat = {}, {}
    fw, fh = 6, 6
    dv_1, dv_2 = "Slope", "Score"
    out_fn = f"Score_x_Slopes.csv"
    folder = f"[gfunc] {rois_name}" if gfunc == "_gfunc" else rois_name
    out_dir = os.path.join(top_dir, "ROI_analysis", 
                           ap_uni, "Corr_with_behav", folder)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)  

    for cond in cond_list: 
        all_cormat[cond], all_annot_mat[cond] = {}, {}

        for roi in roi_list:
            sub_table = slope_table.query("ROI == @roi & Cond == @cond")
            r, p = stats.pearsonr(sub_table[dv_1], sub_table[dv_2], 
                                  alternative="two-sided")
            all_cormat[cond][roi] = r
            all_annot_mat[cond][roi] = format_correlation(r, p)

    cormat = pd.DataFrame(all_cormat)
    cormat.to_csv(os.path.join(out_dir, out_fn))

    # -------------------------

    fig, ax = plt.subplots(figsize=(fw, fh)) 
    sns.set(style='whitegrid', font_scale=1.5)
    heatmap = sns.heatmap(
        cormat, vmin=-1, vmax=1, cmap="RdBu_r", cbar=False, 
        annot=pd.DataFrame(all_annot_mat), fmt = "", 
        linewidth=.5 
    )
    ax.set_xticklabels(cond_list, rotation=0, fontsize=20)
    ax.set_yticklabels(roi_list, rotation=0, fontsize=20)
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, out_fn.replace(".csv", ".png")), 
                format='png', dpi=200)
    plt.close()

In [26]:
smc_table.columns

Index(['Subj', 'ROI', 'Targ', 'Avg_SMC', 'Group_x', 'Design', 'Score',
       'Group_y'],
      dtype='object')

In [27]:
for x, tar in enumerate(targ_namelist):
    print(f"[{x}] {tar}")

[0] fst.pairs
[1] snd.pairs
[2] tpl.nonadj
[3] within.tpl
[4] rare.connect
[5] irrelvant
[6] diff.tpl
[7] tpl.A
[8] tpl.B
[9] tpl.C
[10] tpl.D
[11] Fst
[12] Snd
[13] Trd


In [28]:
## Behavior scores x Similarity changes:

re_do = 1

if re_do:
    all_cormat, all_annot_mat = {}, {}
    dv_1, dv_2 = "Avg_SMC", "Score"
    
    targ_id_list = [3, 0, 1, 2, 4, 5]
    targ_list = [ targ_namelist[x] for x in targ_id_list ] 
    fw, fh = 9, 8
    
    # out_fn = "Score_x_SMC.csv"
    out_fn = f"{rois_name}.csv"
    out_dir = os.path.join(mvpa_top, f"Corr_with_behav{subj_group}", rdm_meas)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)  

    for targ in targ_list: 
        all_cormat[targ], all_annot_mat[targ] = {}, {}

        for roi in roi_list:
            sub_table = smc_table.query("ROI == @roi & Targ == @targ")
            r, p = stats.pearsonr(sub_table[dv_1], sub_table[dv_2], 
                                  alternative="two-sided")
            all_cormat[targ][roi] = r
            all_annot_mat[targ][roi] = format_correlation(r, p)

    cormat = pd.DataFrame(all_cormat)
    cormat.to_csv(os.path.join(out_dir, out_fn))

    # -------------------------

    fig, ax = plt.subplots(figsize=(fw, fh)) 
    sns.set(style='whitegrid', font_scale=1.5)
    heatmap = sns.heatmap(
        cormat, # vmin=-1, vmax=1, 
        cmap=["coolwarm", "RdBu_r"][0], cbar=False, 
        annot=pd.DataFrame(all_annot_mat), fmt = "", 
        linewidth=.5 
    )
    ax.set_xticklabels(targ_list, rotation=45, fontsize=20, ha="right")
#     ax.set_xticklabels(targ_list, rotation=0, fontsize=20)
    ax.set_yticklabels(roi_list, rotation=0, fontsize=20)
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, out_fn.replace(".csv", ".png")), 
                format='png', dpi=200)
    plt.close()

In [ ]:
out_dir

## Specifiy a correlation to plot:

In [ ]:
# ppi_rois

In [ ]:
# plot_spec = 0
# save_plot = 0

# Y_type, R = {
#     1: ("orig", run_list[3]), 
#     2: ("diff", r_diff_list[3])
# }[1]

# roi_1 = "Hippocampus" ###
# roi_2 = ppi_rois[1]
# cond_1 = cond_list[0]
# cond_2 = cond_list[2]
# dv_1 = ["Y", "PPI", "Slope", "Score"][0]
# dv_2 = ["Y", "PPI", "Slope", "Score"][0]

# if plot_spec: 
#     if roi_1 == "Hippocampus":
#         fig_name = f"[R{R}] hipp.{cond_1} x {roi_2}.{cond_2}.png"
#         folder = f"[gfunc] Hipp_x_{ppi_rois_name}" if gfunc == "_gfunc" else f"Hipp_x_{ppi_rois_name}"
#     else:
#         fig_name = f"[R{R}] {roi_1}.{cond_1} x {roi_2}.{cond_2}.png"
#         folder = f"[gfunc] {rois_name}" if gfunc == "_gfunc" else rois_name
        
#     if Y_type == "orig":
#         roi_1_Y = data_table.query("ROI == @roi_1 & "+
#                                    "Cond == @cond_1 & "+
#                                    "Run == @R")[dv_1]
#         roi_2_Y = data_table.query("ROI == @roi_2 & "+
#                                    "Cond == @cond_2 & "+
#                                    "Run == @R")[dv_2]
#     else: # "diff"
#         roi_1_Y = diff_table.query("ROI == @roi_1 & "+
#                                    "Cond == @cond_1 & "+
#                                    "R_diff == @R")[dv_1]
#         roi_2_Y = diff_table.query("ROI == @roi_2 & "+
#                                    "Cond == @cond_2 & "+
#                                    "R_diff == @R")[dv_2]
    
#     x_lab = f"{roi_1}, {cond_1}"
#     y_lab = f"{roi_2}, {cond_2}"
#     DF = pd.DataFrame({x_lab: list(roi_1_Y), 
#                        y_lab: list(roi_2_Y), 
#                        # "ACC": behav_table["Score"], 
#                        "Score": behav_table["Score"]})
    
#     out_dir = os.path.join(top_dir, "ROI_analysis", 
#                            ap_uni, "InterSubj_Corr", folder)
#     if not os.path.exists(out_dir):
#         os.makedirs(out_dir)  
        
#     # -----------------------------

#     fig = plt.figure(num=None, figsize=(5, 5), dpi=500)
#     sns.set(style='whitegrid', font_scale=1.2)

#     g = sns.JointGrid(data=DF, x=x_lab, y=y_lab, 
#                       height=5, ratio=3)
#     g = g.plot_joint(sns.regplot, x_jitter=False, y_jitter=False, 
#                      scatter_kws={'alpha': 0.5, 
#                                   'edgecolor': 'white'}, 
#                      line_kws={'linewidth': 1})
#     g = g.plot_marginals(sns.histplot, kde=True, bins = 15, linewidth=1)
#     g.refline(x=0, color='r')
#     g.refline(y=0, color='r')

#     N = np.shape(DF)[0]
#     r, p = stats.pearsonr(roi_1_Y, roi_2_Y)
#     p_txt = f"p {format_p(p)}" if p < .001 else f"p = {format_p(p)}"
#     plt.suptitle(f"[Run {R}] r = {r:.2f}, {p_txt}, N = {N:.0f}")

#     plt.tight_layout()
    
#     if save_plot:
#         plt.savefig(os.path.join(out_dir, fig_name), 
#                     format='png', dpi=200)

In [ ]:
# ## Color-code participants by score:

# fig = plt.figure(num=None, figsize=(5, 5))
# sns.set(style='whitegrid', font_scale=1.2)

# g = sns.scatterplot(data=DF, x=x_lab, y=y_lab, hue="ACC", 
#                     alpha=0.5, edgecolor="white")
# sns.regplot(data=DF, x=x_lab, y=y_lab, 
#             ax=g.axes, scatter=False)
# g.axhline(0, color='r', linestyle='dashed')
# g.axvline(0, color='r', linestyle='dashed')
# sns.move_legend(
#     g, "center left", bbox_to_anchor=(1, .5), frameon=False
# )

In [ ]:
for x, roi in enumerate(roi_list):
    print(f"[{x}] {roi}")

In [ ]:
## Behavior scores x Similarity changes:

do_plot = 1

out_dir = os.path.join(mvpa_top, f"Corr_with_behav{subj_group}", rdm_meas)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
dv_1, dv_2 = "Avg_SMC", "Score"

if do_plot: 
    for roi in list(roi_list[x] for x in [2, 3, 5]):    
        for targ in list(targ_namelist[x] for x in [0, 1, 3]):
            fig_name = f"[{targ}] {roi}.png"
            
            fig = plt.figure(num=None, figsize=(5, 5), dpi=500)
            sns.set(style='whitegrid', font_scale=1.2)
            
            sub_table = smc_table.query("ROI == @roi & "+
                                        "Targ == @targ")
            g = sns.JointGrid(data=sub_table, x=dv_1, y=dv_2, 
                              height=5, ratio=3)            
            g = g.plot_joint(sns.regplot, 
                             x_jitter=False, y_jitter=False, 
                             scatter_kws={'alpha': 0.5, 
                                          'edgecolor': 'white'}, 
                             line_kws={'linewidth': 1})
            g = g.plot_marginals(sns.histplot, kde=True, bins = 15, linewidth=1)
            g.refline(x=0, color='r')
            g.refline(y=threshold, color='r')
            
            N = np.shape(sub_table)[0]
            r, p = stats.pearsonr(sub_table[dv_1], sub_table[dv_2], 
                                  alternative="two-sided")
            p_txt = f"p {format_p(p)}" if p < .001 else f"p = {format_p(p)}"
            
            g.set_axis_labels(xlabel = f'Mean SMC of "{targ}"')
            plt.suptitle(f"{roi} [r = {r:.2f}, {p_txt}, N = {N:.0f}]")
            plt.tight_layout()
    
            plt.savefig(os.path.join(out_dir, fig_name), 
                        format='png', dpi=200)

## Other try:

In [ ]:
set(ppi_table["Run"])

In [ ]:
for cond in cond_list:
    print(f">> {cond}")
    for roi in ppi_rois:
        values = ppi_table.query("Run == 5 & Cond == @cond & ROI == @roi")["PPI"]
        m = np.mean(values)
        sd = np.std(values)
        t, p = stats.ttest_1samp(values, 0, 
                                 alternative=['two-sided', 'greater'][1])
        print(f"{roi}:\n\t{m:.2f} ({sd:.3f})\n\tt = {t:.2f}, p = {format_p(p)}")

In [ ]:
sns.histplot(ppi_table.query("Run == 5 & Cond == 'Fst' & ROI == 'Fusiform'")["PPI"])